<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Исследование-надежности-заемщиков" data-toc-modified-id="Исследование-надежности-заемщиков-1">Исследование надежности заемщиков</a></span></li><li><span><a href="#Импортирование-библиотек" data-toc-modified-id="Импортирование-библиотек-2">Импортирование библиотек</a></span></li><li><span><a href="#Изучение-данных" data-toc-modified-id="Изучение-данных-3">Изучение данных</a></span></li><li><span><a href="#Предобработка-данных" data-toc-modified-id="Предобработка-данных-4">Предобработка данных</a></span></li><li><span><a href="#Проверка-гипотез" data-toc-modified-id="Проверка-гипотез-5">Проверка гипотез</a></span></li><li><span><a href="#Общий-вывод" data-toc-modified-id="Общий-вывод-6">Общий вывод</a></span></li></ul></div>

# Исследование надежности заемщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

# Импортирование библиотек

In [1]:
import pandas as pd

try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

# Изучение данных

*Рассмотрим корректность названий столбцов.*

In [2]:
data.columns

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')

Столбцы не содержат пробелов, все знаки в одном регистре и на одном языке.

*Рассмотрим общую информацию.*

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Заметим, что присутствуют два столбца содержащих пропущенные значения. Типы столбцов соответствуют информации хранящихся в них

*Рассмотрим первые 15 строк датафрейма.*

In [4]:
data.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


# Предобработка данных

Выведем количество пропущенных значений для каждого столбца.

In [5]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Можем наблюдать пропуски в столбцах 'days_employed' и 'total_income'.

На сумму дохода('income_type') сильнее всего влияет тип занятости, поэтому заполним пропуски в этом столбце медианным значением по каждому типу из столбца income_type.

In [6]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

*Обработаем аномальные значения в столбце 'days_employed' заменив все отрицательные значения положительными.*

In [7]:
data['days_employed'] = data['days_employed'].abs()

*Для каждого типа занятости выведем медианное значение трудового стажа('days_employed') в днях.*

In [8]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставим их как есть. Тем более этот столбец не понадобится вам для исследования.

*Выведем перечень уникальных значений столбца 'children'.*

In [9]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

*В столбце 'children' есть два аномальных значения. Удалим строки, в которых встречаются такие аномальные значения.*

In [10]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

*Ещё раз выведем перечень уникальных значений столбца 'children', чтобы убедиться, что ненужные значения удалены.**

In [11]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

*Заполним пропуски в столбце 'days_employed' медианными значениями по каждому типу занятости 'income_type'.*

In [12]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

*Убедимся, что все пропуски заполнены.*

In [13]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

*Заменим вещественный тип данных в столбце 'total_income' на целочисленный.*

In [14]:
data['total_income'] = data['total_income'].astype(int)

*Обработаем неявные дубликаты в столбце 'education'. Приведём их к нижнему регистру.*

In [15]:
data['education'] = data['education'].str.lower()

*Выведем на экран количество дубликатов в данных.*

In [16]:
data.duplicated().sum()

71

*Удалим дубликаты.*

In [17]:
data = data.drop_duplicates()

Cоздадим в датафрейме столбец 'total_income_category' с категориями:

- 0–30000 — 'E'
- 30001–50000 — 'D'
- 50001–200000 — 'C'
- 200001–1000000 — 'B'
- 1000001 и выше — 'A'

In [18]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [19]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

*Выведем на экран уникальные значения из столбца 'purpose'.*

In [20]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Cоздадим в датафрейме столбец 'purpose_category', в который войдут следующие категории:

- 'операции с автомобилем'
- 'операции с недвижимостью'
- 'проведение свадьбы'
- 'получение образования'


In [21]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [22]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

# Проверка гипотез

**Есть ли зависимость между количеством детей и возвратом кредита в срок?**

In [23]:
children = data.pivot_table(index=['children'], values='debt', aggfunc=('mean','sum','count'))

def percent(i):
    return '{0:.2%}'.format(i)

children['mean'] = children['mean'].apply(percent)

children = children.rename(columns={
    'sum': 'сумма',
    'count': 'количество',
    'mean': 'среднее'
})
children

,количество,среднее,сумма
children,,,
0,14091.0,7.54%,1063.0
1,4808.0,9.23%,444.0
2,2052.0,9.45%,194.0
3,330.0,8.18%,27.0
4,41.0,9.76%,4.0
5,9.0,0.00%,0.0


Вывод: Взглянув на таблицу мы увидим, что у заёмщиков с 3,4 и 5 детьми недостаточное количество данных в сравненнии другими категориями.

- 14091 - бездетные 
- 4808 - 1 ребёнок
- 2052 - 2 ребёнка
- 330 - 3 ребёнка
- 41 - 4 ребёнка
- 9 - 5 детей

Отбросив малочисленные группы и продолжив анализ оставив только три самые большие выборки с одним, двумя и отсутствием детей, можем заметить, что бездетные заёмщики(7.54%) с большей вероятностью возвращают кредиты в сравнении с гражданами с 1(9.23%) и 2(9.45%) детьми.

**Есть ли зависимость между семейным положением и возвратом кредита в срок?**

In [24]:
family = data.pivot_table(index=['family_status'], values='debt', aggfunc=('mean','sum','count'))

def percent(i):
    return '{0:.2%}'.format(i)

family['mean'] = family['mean'].apply(percent)

family = family.rename(columns={
    'sum': 'сумма',
    'count': 'количество',
    'mean': 'среднее'
})
family

,количество,среднее,сумма
family_status,,,
Не женат / не замужем,2796.0,9.76%,273.0
в разводе,1189.0,7.06%,84.0
вдовец / вдова,951.0,6.62%,63.0
гражданский брак,4134.0,9.31%,385.0
женат / замужем,12261.0,7.56%,927.0


Глядя на таблицу сложно определить зависимость от семейного положения. Углубимся в анализ и разделим людей на 2 группы: 'состоявшие или состоящие в зарегистрированном браке' и 'не состоявшие в зарегистрированном браке'.

In [25]:
def family_groups(i):
    try:
        if 'Не женат / не замужем' in i or 'гражданский брак' in i:
            return 'не состоявшие в зарегистрированном браке'
        elif 'в разводе' in i or 'вдовец / вдова' in i or 'женат / замужем' in i:
            return 'состоявшие или состоящие в зарегистрированном браке'
    except:
        return 'некорректное значение'
    
data['family_groups'] = data['family_status'].apply(family_groups)    
    

Создадим таблицу категоризированных данных.

In [26]:
family = data.pivot_table(index=['family_groups'], values='debt', aggfunc=('mean','sum','count'))

def percent(i):
    return '{0:.2%}'.format(i)

family['mean'] = family['mean'].apply(percent)

family = family.rename(columns={
    'sum': 'сумма',
    'count': 'количество',
    'mean': 'среднее'
})

family

,количество,среднее,сумма
family_groups,,,
не состоявшие в зарегистрированном браке,6930.0,9.49%,658.0
состоявшие или состоящие в зарегистрированном браке,14401.0,7.46%,1074.0


Вывод: В таблице мы можем наблюдать разницу в 2.03% между 'не состоявшие в зарегистрированном браке' и 'состоявшие или состоящие в зарегистрированном браке', что в свою очередь является существенным показателем. Следовательно мы можем подтвердить зависимость погашения кредита от семейного положения.

**Есть ли зависимость между уровнем дохода и возвратом кредита в срок?**

In [27]:
income = data.pivot_table(index=['total_income_category'], values='debt', aggfunc=['sum', 'count', 'mean'])
income.columns = ['сумма', 'количество', 'среднее']
income['среднее'] = [f'{x:.2%}' for x in income['среднее']]

income

,сумма,количество,среднее
total_income_category,,,
A,2,25,8.00%
B,354,5014,7.06%
C,1353,15921,8.50%
D,21,349,6.02%
E,2,22,9.09%


Вывод: Ознакомившись с таблицей мы можем наблюдать две категории граждан: 'B' и 'C' с наибольшим количеством займов. Остальные группы: 'A','D','E' можем отбросить из анализа, так как их количество несущественно. Обратившись к таблице мы увидим существенное отличие в количестве кредитуемых граждан: 'С'- 15921 и 'B'- 5014. Помимо этого заёмшики категории 'C' реже заёмщиков категории 'B' оплачивают задолженность вовремя, разница составляет 1.44%. Следовательно мы наблюдаем зависимость от уровня дохода для погашения кредита в срок.

**Как разные цели кредита влияют на его возврат в срок?**

In [28]:
purpose_group = data.pivot_table(index=['purpose_category'], values='debt', aggfunc=['sum', 'count', 'mean'])
purpose_group.columns = ['сумма', 'количество', 'среднее']
purpose_group['среднее'] = [f'{x:.2%}' for x in purpose_group['среднее']]

purpose_group

,сумма,количество,среднее
purpose_category,,,
операции с автомобилем,400,4279,9.35%
операции с недвижимостью,780,10751,7.26%
получение образования,369,3988,9.25%
проведение свадьбы,183,2313,7.91%


Вывод: Посмотрев на таблицу мы обнаружим, что задолженность по операциям с недвижимостью наименьшая и равна 7.26%. Так же интересной особенностью категории с недвижимостью является большое количество займов, равное 10751, возможно это связано с государственными льготами. Операции с автомобилями имеют самую высокую задолженность и равны 9.35%. Практически аналогичный показатель с автомобильными операциями имеет категория взявшая кредит на получение высшего образования - 9.25%.

# Общий вывод

Было проведено исследование, чтобы выявить влияние разных факторов на факт погашения кредита в скрок. Входные данные - статистика о платёжеспособности клиентов. Результаты исследования будут учтены при построении модели кредитного скоринга - специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

Преред проведением исследования были поставлены несколько гипотез:

- Гипотеза 1: количество детей влияет на возврат кредита в срок;
- Гипотеза 2: семейное положение влияет на возврат кредита в срок;
- Гипотеза 1: уровень дохода влияет на возврат кредита в срок;
- Гипотеза 1: цели кредита влияют на возврат кредита в срок.

Гипотеза 1 частично подтверждена: для более точных выводов необходимо больше данных по клиентам с 3,4 и 5 детьми.

- 7.54% - доля просроченых кредитов у заёмщиков без детей;
- 9.23% - доля просроченых кредитов у заёмщиков у заёмщиков с 1 ребёнком;
- 9.45% - доля просроченых кредитов у заёмщиков с 2 детьми.

Рекомендация: провести дополнительное исследование на большей выборке данных.

Гипотеза 2 подверждена.

У не женатых клиентов(9.76%) и тех у кого официально не зарегестрирован брак(9.31%), выше вероятность неуплаты по кредиту. Те кто в разводе (7.06%), женат/замужем(7.56%) или вдовец/вдова(6.62%), меньше вероятность не выполнить обязательства по кредиту.

Из даннного исследования мы можем сделать вывод, что люди состоявшие или состоящие в зарегистрированном браке чаще выплачивают задолженнности по кредиту(7.46%), в сравнении с категорией никогда не состоявшей в зарегестрированном браке(9.49%).

Гипотеза 3 частично подверждена: для более точных выводов необходимо больше данных по клиентам с уровнем дохода из категорий: 'D'(от 30001 до 50000),'E'(до 30000),'A'(от 1000001).

- 7.06% - доля просроченных кредитов у заёмщиков с уровнем дохода от 200001 до 1000000;
- 8.50% - доля просроченных кредитов у заёмщиков с уровнем дохода от 50001 до 200000.

Рекомендация: провести дополнительное исследование на большей выборке данных.

Гипотеза 4 подтверждена:

- 7.26% - доля просроченных кредитов,целью которых были операции с недвижимостью;
- 7.91% - доля просроченных кредитов,целью которых было проведение свадьбы;
- 9.35% - доля просроченных кредитов,целью которых были операции с автомобилями;
- 9.25% - доля просроченных кредитов,целью которых было получени образования.

Из даннного исследования мы можем сделать вывод, что люди берущие кредит на оперции с недвижимостью более ответственно подходят к задолженности(7.26%). Так же люди берущие кредит для проведения свадьбы более ответственно подходят к выполнению своих обязательств(7.91%) в сравнениями с другими категориями: 'операции с автомобилями'(9.35%), 'получени образования'(9.25%).

Общие рекомендации: 
- 1.Специалистам,которые занимаются построением моделикредитного скоринга, можно использовать зависимости из второй и четвёртой гипотезы(семейное положение и цели кредита). 
- 2.Необходимо собрать более полную выборку для повторной проверки первой и третьей гипотез.